In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import requests
import json
from census import Census

In [3]:
# read in Fast Food & Obesity csv files - Anne

#connect to Census API - Julia 

#convert Census API zip code into state

#merge tables by state 

#create data frame including state, fast food chain, obesity rate

#data frame population and state



In [37]:
### Data Cleanup 
# Census data
# Census API Key

url = 'http://api.census.gov/data/2020/dec/pl?get=NAME,H1_001N&for=state:*'

from config import api_key
c = Census(api_key, year=2021)

census_data = requests.get(url).json()

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

population_by_state = census_pd.rename(columns={"NAME": "State", 
                                      "H1_001N": "Total Population"})
population_by_state

,0,1,2
0,NAME,H1_001N,state
1,Alabama,2288330,01
2,Alaska,326200,02
3,Arizona,3082000,04
4,Arkansas,1365265,05
5,California,14392140,06
6,Colorado,2491404,08
7,Connecticut,1530197,09
8,Delaware,448735,10
9,District of Columbia,350364,11


In [ ]:
### Data Cleanup 
# Obesity data

# read in csv file - Anne